# tools

#### query.py

In [1]:
from graphly.tools.query import get_sparql_type

select = """
PREFIX dbo: <http://dbpedia.org/ontology/>
SELECT ?city ?country
WHERE {
  ?city a dbo:City ;
        dbo:country ?country .
}
LIMIT 5
"""
insert = """
PREFIX ex: <http://example.org/>
INSERT DATA {
  ex:Paris a ex:City ;
           ex:country ex:France .
}
"""
delete = """
PREFIX ex: <http://example.org/>
DELETE DATA {
  ex:Paris a ex:City ;
           ex:country ex:France .
}
"""
construct = """
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT {
  ?city a dbo:City ;
        dbo:country ?country .
}
WHERE {
  ?city a dbo:City ;
        dbo:country ?country .
}
LIMIT 5
"""
clear = """
PREFIX ex: <http://example.org/>
CLEAR GRAPH <http://example.org/myGraph>
"""

assert get_sparql_type(select) == 'SELECT'
assert get_sparql_type(insert) == 'INSERT'
assert get_sparql_type(delete) == 'DELETE'
assert get_sparql_type(construct) == 'CONSTRUCT'
assert get_sparql_type(clear) == 'CLEAR'

#### triple.py

In [2]:
from graphly.schema import Prefixes, Prefix
from graphly.tools import prepare_triple

prefixes = Prefixes([Prefix('base', 'http://example')])

assert prepare_triple(('base:123', 'http://example/label', 'hello world'), prefixes.shorts()) == "base:123 <http://example/label> 'hello world' ."
assert prepare_triple(('base:123', 'http://example/label', 3), prefixes.shorts()) == "base:123 <http://example/label> 3 ."

#### uri.py

In [4]:
from graphly.schema import Prefixes, Prefix, Resource
from graphly.tools import prepare

prefixes = Prefixes([Prefix('base', 'http://example')])

assert prepare(None) == None
assert prepare(3) == '3'
assert prepare("3") == "'3'"
assert prepare("http://example") == "<http://example>"
assert prepare("base:hello", prefixes.shorts()) == "base:hello"
assert prepare("unknown:hello", prefixes.shorts()) == "'unknown:hello'"

r = Resource("test:123", "Name")
assert str(r) == 'test:123'

# schema

#### prefix.py

In [5]:
import json
from graphly.schema import Prefix

p = Prefix('short', 'long')

ex = Prefix('ex', 'http://example.com/')

assert ex.to_sparql() == "PREFIX ex: <http://example.com/>"
assert ex.to_turtle() == "@prefix ex: <http://example.com/> ."
assert ex.shorten("http://example.com/i123") == "ex:i123"
assert ex.lengthen("ex:i123") == "http://example.com/i123"
assert json.dumps(ex.to_dict()) == json.dumps(Prefix.from_dict(ex.to_dict()).to_dict())

#### prefixes.py

In [6]:
ps = Prefixes()
ps = Prefixes([Prefix('ex', 'http://example.com/')])
assert ps.has('rdf') == False
assert ps.has('ex') == True
assert ps.shorten("http://example.com/i123") == "ex:i123"
assert ps.lengthen("ex:i123") == "http://example.com/i123"

test = Prefix('test', 'http://test.com/')
ps.add(test)
assert ps.has('test') == True
assert ps.find('ex').short == 'ex'
assert len(ps) == 2

#### resource.py

In [1]:
import json
from graphly.schema import Resource

r0 = Resource("test")

r1 = Resource("A String", class_uri="xsd:string", resource_type='literal')
assert r1.get_text() == "A String"
assert r1.get_text(comment=True) == "A String"
assert json.dumps(r1.to_dict()) == json.dumps(Resource.from_dict(r1.to_dict()).to_dict())


r2 = Resource(3, class_uri="xsd:integer", resource_type="literal")
assert r2.get_text() == "3"
assert r2.get_text(comment=True) == "3"
assert json.dumps(r2.to_dict()) == json.dumps(Resource.from_dict(r2.to_dict()).to_dict())

r3 = Resource("base:i123", "Entity Label", "Entity comment", class_uri="crm:E21")
assert r3.get_text() == "Entity Label"
assert r3.get_text(comment=True) == "Entity Label: Entity comment"
assert json.dumps(r3.to_dict()) == json.dumps(Resource.from_dict(r3.to_dict()).to_dict())

#### property.py

In [2]:
import json
from graphly.schema import Resource, Property

class1 = Resource("uri:class1", "Class1")
predicate = Resource("uri:predicate", "predicate name", "predicate comment", "owl:Property")
class2 = Resource("uri:class2", "Class2")
class3 = Resource("uri:class3", "Class3")

p0 = Property("uri:predicate", "predicate name", "predicate comment")
assert p0.get_key() == "unknown-uri:predicate-unknown"
assert p0.is_mandatory() == False
assert json.dumps(p0.to_dict()) == json.dumps(Property.from_dict(p0.to_dict()).to_dict())

p1 = Property("uri:predicate", "predicate name", "predicate comment", class1, class2, class3, 1, 0, 3)
assert p1.get_key() == "uri:class1-uri:predicate-uri:class2"
assert p1.is_mandatory() == False
assert json.dumps(p1.to_dict()) == json.dumps(Property.from_dict(p1.to_dict()).to_dict())

p2 = Property("uri:predicate", "predicate name", "predicate comment", class1, None, class3, 1, 1, 3)
assert p2.get_key() == "uri:class1-uri:predicate-unknown"
assert p2.is_mandatory() == True
assert json.dumps(p2.to_dict()) == json.dumps(Property.from_dict(p2.to_dict()).to_dict())

#### statement.py

In [3]:
import json
from graphly.schema import Resource, Property, Statement, Prefixes, Prefix

prefixes = Prefixes([
    Prefix('owl', 'http://example/owl'),
    Prefix('crm', 'http://example/crm'),
])

c1 = Resource("http://example/person", "Person", class_uri="owl:Class")
p1 = Resource("http://example/p1", "Person1", class_uri="crm:E21")
p2 = Resource("http://example/p2", "Person2", class_uri="crm:E21")
prop = Property("http://example/predicate1", "Predicate1", "", c1, c1, c1)

s0 = Statement(p1, prop, p2)
assert json.dumps(s0.to_dict()) == json.dumps(Statement.from_dict(s0.to_dict()).to_dict())

#### ontology.py

In [4]:
from graphly.schema import Ontology

o = Ontology("My Ontology", "onto", "http://my-ontology.org/")

assert o.name == "My Ontology"
assert o.prefix.short == "onto"
assert o.prefix.long == "http://my-ontology.org/"

#### sparql.py

In [5]:
# schema/sparql.py

from graphly.schema import Sparql

z = Sparql('url', 'user', 'password')


# sparql

#### allegrograph.py

In [ ]:
import os
import dotenv
dotenv.load_dotenv()
from graphly.sparql import Allegrograph
from graphly.schema import Prefix

endpoint = Allegrograph(os.getenv('GRAPHLY_TEST_ALLEGROGRAPH_URL'), os.getenv('GRAPHLY_TEST_ALLEGROGRAPH_USERNAME'), os.getenv('GRAPHLY_TEST_ALLEGROGRAPH_PASSWORD'))
endpoint.run('CLEAR ALL')

endpoint.insert(('http://example/1', 'http://example/2', 'http://example/3'))
result1 = endpoint.run('select * where { ?s ?p ?o . }')
assert result1[0]['s'] == 'http://example/1'
assert result1[0]['p'] == 'http://example/2'
assert result1[0]['o'] == 'http://example/3'

endpoint.delete(('http://example/1', 'http://example/2', 'http://example/3'))
endpoint.insert(('http://example/1', 'http://example/2', 'http://example/4'))
result2 = endpoint.run('select * where { ?s ?p ?o . }')
assert result2[0]['s'] == 'http://example/1'
assert result2[0]['p'] == 'http://example/2'
assert result2[0]['o'] == 'http://example/4'

endpoint.run('CLEAR ALL')
n_quad = """
<http://example/10> <http://example/20> <http://example/30> .
<http://example/40> <http://example/50> <http://example/60> <http://graph> .
"""
turtle = f"""
{Prefix("test", "http://test/").to_turtle()}
test:123 test:456 test:789 .
test:12 test:45 <http://example/78> .
test:12 test:99 'hello world' .
"""
endpoint.upload_nquads(n_quad)
endpoint.upload_turtle(turtle, 'http://graph/turtle')
dump_lines = endpoint.dump().split('\n')
assert "<http://example/10> <http://example/20> <http://example/30> ." in dump_lines
assert "<http://example/40> <http://example/50> <http://example/60> <http://graph> ." in dump_lines
assert "<http://test/123> <http://test/456> <http://test/789> <http://graph/turtle> ." in dump_lines
assert "<http://test/12> <http://test/45> <http://example/78> <http://graph/turtle> ." in dump_lines
assert "<http://test/12> <http://test/99> 'hello world' <http://graph/turtle> ." in dump_lines or \
       '<http://test/12> <http://test/99> "hello world" <http://graph/turtle> .'


#### fuseki.py

In [6]:
import os
import dotenv
dotenv.load_dotenv()
from graphly.sparql import Fuseki
from graphly.schema import Prefix

endpoint = Fuseki(os.getenv('GRAPHLY_TEST_FUSEKI_URL'), os.getenv('GRAPHLY_TEST_FUSEKI_USERNAME'), os.getenv('GRAPHLY_TEST_FUSEKI_PASSWORD'))
endpoint.run('CLEAR ALL')

endpoint.insert(('http://example/1', 'http://example/2', 'http://example/3'))
result1 = endpoint.run('select * where { ?s ?p ?o . }')
assert result1[0]['s'] == 'http://example/1'
assert result1[0]['p'] == 'http://example/2'
assert result1[0]['o'] == 'http://example/3'

endpoint.delete(('http://example/1', 'http://example/2', 'http://example/3'))
endpoint.insert(('http://example/1', 'http://example/2', 'http://example/4'))
result2 = endpoint.run('select * where { ?s ?p ?o . }')
assert result2[0]['s'] == 'http://example/1'
assert result2[0]['p'] == 'http://example/2'
assert result2[0]['o'] == 'http://example/4'

endpoint.run('CLEAR ALL')
n_quad = """
<http://example/10> <http://example/20> <http://example/30> .
<http://example/40> <http://example/50> <http://example/60> <http://graph> .
"""
turtle = f"""
{Prefix("test", "http://test/").to_turtle()}
test:123 test:456 test:789 .
test:12 test:45 <http://example/78> .
test:12 test:99 'hello world' .
"""
endpoint.upload_nquads(n_quad)
endpoint.upload_turtle(turtle, 'http://graph/turtle')
dump_lines = endpoint.dump().split('\n')
assert "<http://example/10> <http://example/20> <http://example/30> ." in dump_lines
assert "<http://example/40> <http://example/50> <http://example/60> <http://graph> ." in dump_lines
assert "<http://test/123> <http://test/456> <http://test/789> <http://graph/turtle> ." in dump_lines
assert "<http://test/12> <http://test/45> <http://example/78> <http://graph/turtle> ." in dump_lines
assert "<http://test/12> <http://test/99> 'hello world' <http://graph/turtle> ." in dump_lines


> Uploading 1 lines (0 / 3)
> Uploading (0 / 4)


#### graphdb.py

In [ ]:
import os
import dotenv
dotenv.load_dotenv()
from graphly.sparql import GraphDB
from graphly.schema import Prefix

endpoint = GraphDB(os.getenv('GRAPHLY_TEST_GRAPHDB_URL'), os.getenv('GRAPHLY_TEST_GRAPHDB_USERNAME'), os.getenv('GRAPHLY_TEST_GRAPHDB_PASSWORD'))
endpoint.run('CLEAR ALL')

endpoint.insert(('http://example/1', 'http://example/2', 'http://example/3'))
result1 = endpoint.run('select * where { ?s ?p ?o . }')
found = False
for triple in result1:
    if triple['s'] == 'http://example/1' and triple['p'] == 'http://example/2' and triple['o'] == 'http://example/3':
        found = True
        break
assert found

endpoint.delete(('http://example/1', 'http://example/2', 'http://example/3'))
endpoint.insert(('http://example/1', 'http://example/2', 'http://example/4'))
result2 = endpoint.run('select * where { ?s ?p ?o . }')
found = False
for triple in result2:
    if triple['s'] == 'http://example/1' and triple['p'] == 'http://example/2' and triple['o'] == 'http://example/4':
        found = True
        break
assert found

endpoint.run('CLEAR ALL')
n_quad = """
<http://example/10> <http://example/20> <http://example/30> .
<http://example/40> <http://example/50> <http://example/60> <http://graph> .
"""
turtle = f"""
{Prefix("test", "http://test/").to_turtle()}
test:123 test:456 test:789 .
test:12 test:45 <http://example/78> .
test:12 test:99 'hello world' .
"""
endpoint.upload_nquads(n_quad)
endpoint.upload_turtle(turtle, 'http://graph/turtle')
dump_lines = endpoint.dump().split('\n')
assert "<http://example/10> <http://example/20> <http://example/30> ." in dump_lines
assert "<http://example/40> <http://example/50> <http://example/60> <http://graph> ." in dump_lines
assert "<http://test/123> <http://test/456> <http://test/789> <http://graph/turtle> ." in dump_lines
assert "<http://test/12> <http://test/45> <http://example/78> <http://graph/turtle> ." in dump_lines
assert "<http://test/12> <http://test/99> 'hello world' <http://graph/turtle> ." in dump_lines or \
       '<http://test/12> <http://test/99> "hello world" <http://graph/turtle> .'


# schema

#### graph.py

In [7]:
from os import getenv
from dotenv import load_dotenv
from graphly.schema import Graph, Prefix, Prefixes
from graphly.sparql import Fuseki
load_dotenv()

prefix = Prefix('test', 'http://example.com/')
prefixes = Prefixes([prefix])
sparql = Fuseki(getenv('GRAPHLY_TEST_FUSEKI_URL'), getenv('GRAPHLY_TEST_FUSEKI_USERNAME'), getenv('GRAPHLY_TEST_FUSEKI_PASSWORD'))

g = Graph(sparql)

assert g.sparql_begin == ''
assert g.sparql_end == ''

g.run('CLEAR ALL')
triples = [("test:i1", "test:p1", "http://example.com/i2"), ("test:i1", "test:p2", "http://example.com/i3")]
g.insert(triples, prefixes)
g.delete(("test:i1", "test:p1", "?o"), prefixes)    
result = g.run('select ?s ?p ?o where { ?s ?p ?o .}')
assert len(result) == 1
assert result[0]['o'] == 'http://example.com/i3'

result = g.dump_dict(prefixes)
assert len(result) == 1
assert result[0]['o'] == 'test:i3'

result = g.dump_turtle(prefixes)
assert result.endswith('test:i1 test:p2 test:i3 .\n')

result = g.dump_nquad(prefixes)
assert result == "<http://example.com/i1> <http://example.com/p2> <http://example.com/i3> .\n"

g.run('CLEAR ALL')
turtle = """
@prefix test: <http://example.com/>. 

test:e1 test:p1 test:e2;
        test:p2 test:e3;
"""
g.upload_turtle(turtle)
result = g.dump_dict(prefixes)
assert len(result) == 2
assert result[0]['o'] == 'test:e3'
assert result[0]['p'] == 'test:p2'
assert result[1]['o'] == 'test:e2'
assert result[1]['p'] == 'test:p1'



g = Graph(sparql, "test:graph", prefixes)

assert g.sparql_begin == 'GRAPH test:graph {'
assert g.sparql_end == '}'
g.run('CLEAR ALL', prefixes)
triples = [("test:i1", "test:p1", "http://example.com/i2"), ("test:i1", "test:p2", "http://example.com/i3")]
g.insert(triples, prefixes)
g.delete(("test:i1", "test:p1", "?o"), prefixes)    
result = g.run('select ?s ?p ?o where { graph test:graph { ?s ?p ?o .}}', prefixes)
assert len(result) == 1
assert result[0]['o'] == 'test:i3'

result = g.dump_dict(prefixes)
assert len(result) == 1
assert result[0]['o'] == 'test:i3'

result = g.dump_turtle(prefixes)
assert result.endswith('test:i1 test:p2 test:i3 .\n')

result = g.dump_nquad(prefixes)
assert result == "<http://example.com/i1> <http://example.com/p2> <http://example.com/i3> <http://example.com/graph> .\n"

g.run('CLEAR ALL')
turtle = """
@prefix test: <http://example.com/>. 

test:e1 test:p1 test:e2;
        test:p2 test:e3;
"""
g.upload_turtle(turtle)
result = g.dump_dict(prefixes)
assert len(result) == 2
assert result[0]['o'] == 'test:e3'
assert result[0]['p'] == 'test:p2'
assert result[1]['o'] == 'test:e2'
assert result[1]['p'] == 'test:p1'

> Uploading (0 / 4)
> Uploading (0 / 4)


#### model.py

In [8]:
from os import getenv
from dotenv import load_dotenv
from graphly.schema import Model, Graph, Prefix, Prefixes
from graphly.sparql import Fuseki
load_dotenv()

prefix = Prefix('test', 'http://example.com/')
prefixes = Prefixes([prefix])
sparql = Fuseki(getenv('GRAPHLY_TEST_FUSEKI_URL'), getenv('GRAPHLY_TEST_FUSEKI_USERNAME'), getenv('GRAPHLY_TEST_FUSEKI_PASSWORD'))
graph = Graph(sparql, '', prefixes)
graph.run('CLEAR ALL', prefixes)
triples = [
    ('test:e1', 'test:p1', 'test:c1'),
    ('test:c1', 'test:p2', 'Class1'),
    ('test:e1', 'test:p2', 'E1 label'),
    ('test:e1', 'test:p3', 'E1 Comment'),
    ('test:e1', 'test:p4', 'Something'),
]
graph.insert(triples, prefixes)

model = Model(type_property='test:p1', label_property='test:p2', comment_property='test:p3')
model.update(graph, prefixes)

assert model.classes[0].resource_type == 'entity'
assert model.classes[0].uri == 'test:c1'
assert model.classes[0].label == 'Class1'
assert model.classes[0].class_uri == 'owl:Class'

assert model.properties[0].domain.uri == "test:c1"
assert model.properties[0].uri == "test:p4"
assert model.properties[0].range.uri == "xsd:string"

# models

#### shacl.py

In [9]:
from os import getenv, environ
from dotenv import load_dotenv
from graphly.models import SHACL
from graphly.schema import Graph, Prefix, Prefixes
from graphly.sparql import Fuseki
load_dotenv()

environ['GRAPHLY_MODE'] = 'normal'

prefix_test = Prefix('test', 'http://example.com/')
prefix_shacl = Prefix('sh', 'http://www.w3.org/ns/shacl#')
prefix_xsd = Prefix('xsd', 'http://www.w3.org/2001/XMLSchema#')
prefix_crm = Prefix('crm', 'http://www.cidoc-crm.org/cidoc-crm/')
prefix_rdf = Prefix('rdf', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#')
prefixes = Prefixes([prefix_test, prefix_shacl, prefix_crm, prefix_xsd, prefix_rdf])
sparql = Fuseki(getenv('GRAPHLY_TEST_FUSEKI_URL'), getenv('GRAPHLY_TEST_FUSEKI_USERNAME'), getenv('GRAPHLY_TEST_FUSEKI_PASSWORD'))
graph = Graph(sparql, 'test:shacl', prefixes)
graph.run('CLEAR ALL', prefixes)

shacl = """
@prefix sdh-shacl: <https://sdhss.org/shacl/profiles/>. 
@prefix sh: <http://www.w3.org/ns/shacl#> .  
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: 	<http://www.w3.org/2000/01/rdf-schema#> .  
@prefix sdh-short: <https://sdhss.org/ontology/shortcuts/>.
@prefix sdh: <https://sdhss.org/ontology/core/>.
@prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>. 

    sdh-shacl:crm_E21_Shape a sh:NodeShape ; 
        sh:targetClass crm:E21 ;
        sh:name "Person" ;

        sh:property [
            sh:path rdfs:label ;
            sh:name "Label";
            sh:datatype xsd:string;
            sh:order 1;
            sh:minCount 1;
            sh:maxCount 1;
        ];

        sh:property [
            sh:path rdfs:comment ;
            sh:name "Description";
            sh:datatype rdf:HTML;
            sh:order 2;
            sh:minCount 1;
        ];  
         
        sh:property [
            sh:path sdh-short:P2 ;
            sh:name "has birth date";
            sh:datatype xsd:string;
            sh:maxCount 1;
            sh:order 3; # verify what happens if new properties
        ];                   
        . 

    sdh-shacl:crm_E67_Shape a sh:NodeShape ;
        sh:targetClass crm:E67 ;
        sh:name "Birth" ;

        sh:property [
            sh:path rdfs:label ;
            sh:name "Label";
            sh:datatype xsd:string;
            sh:order 1;
            sh:maxCount 1;
        ];

        sh:property [
            sh:path rdfs:comment ;
            sh:name "Description";
            sh:datatype rdf:HTML;
            sh:order 2;
        ];  
            
        sh:property [
            sh:path crm:P8 ;
            sh:name "took place on or within";
            sh:class sdh:C17 ;
            sh:order 3;
        ];
                        
        sh:property [
            sh:path crm:P98 ;
            sh:name "brought into life";
            sh:class crm:E21 ;
            sh:order 4;
        ];
                        
        sh:property [
            sh:path sdh:P6 ;
            sh:name "took place at";
            sh:class sdh:C13 ;
            sh:order 5;
        ];
                    
        sh:property [
            sh:path sdh-short:P1 ;
            sh:name "at some time within";
            sh:datatype xsd:string;
            sh:order 6;
            sh:maxCount 1;
        ];

        sh:property [
            sh:path sdh-short:P4 ;
            sh:name "begins on";
            sh:datatype xsd:string;
            sh:order 7;
            sh:maxCount 1;
        ];

        sh:property [
            sh:path sdh-short:P7 ;
            sh:name "ends on";
            sh:datatype xsd:string;
            sh:order 8;
            sh:maxCount 1;
        ];

        sh:property [
            sh:path sdh-short:P3 ;
            sh:name "begins after";
            sh:datatype xsd:string;
            sh:order 9;
            sh:maxCount 1;
        ];

        sh:property [
            sh:path sdh-short:P5 ;
            sh:name "begins before";
            sh:datatype xsd:string;
            sh:order 10;
            sh:maxCount 1;
        ];

        sh:property [
            sh:path sdh-short:P6 ;
            sh:name "ends after";
            sh:datatype xsd:string;
            sh:order 11;
            sh:maxCount 1;
        ];

        sh:property [
            sh:path sdh-short:P8 ;
            sh:name "ends before";
            sh:datatype xsd:string;
            sh:order 12;
            sh:maxCount 1;
        ];   
        . 
"""

graph.upload_turtle(shacl)

model_shacl = SHACL()
model_shacl.update(graph, prefixes)

classes_uris = [c.uri for c in model_shacl.classes]
prop_uris = [p.uri for p in model_shacl.properties]

assert 'crm:E21' in classes_uris
assert 'crm:E67' in classes_uris
assert 'xsd:string' in classes_uris
assert 'rdf:HTML' in classes_uris

assert 'crm:P8' in prop_uris
assert 'crm:P98' in prop_uris
assert 'https://sdhss.org/ontology/core/P6' in prop_uris

crm_P98 = next(p for p in model_shacl.properties if p.uri == 'crm:P98')

assert crm_P98.card_of.uri == 'crm:E67'
assert crm_P98.domain.uri == 'crm:E67'
assert crm_P98.range.uri == 'crm:E21'

> Uploading (0 / 128)


# ontologies

#### crm.py

In [10]:
from graphly.ontologies import CRM

assert CRM.E21_Person.uri == 'crm:E21'
assert CRM.P98_Brought_Into_Life.uri == 'crm:P98'

#### sdh.py

In [11]:
from graphly.ontologies import SDH

assert SDH.C13_Geographical_Place.uri == "sdh:C13"
assert SDH.P6_Took_Place_At.uri == 'sdh:P6'

---